In [ ]:
import time

# import webdriver
from selenium import webdriver


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support.ui import WebDriverWait

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException


# import Action chains 
from selenium.webdriver.common.action_chains import ActionChains

import pyautogui
# need pynput too bc when attempting to write using pyautogui, it also simultaneously opens my emoji keyboard and activates my mic??? 
from pynput.keyboard import Key, Controller
keyboard = Controller()

import os
from PIL import Image
from PIL.PngImagePlugin import PngInfo

In [ ]:
driver = webdriver.Chrome()

for pg in range(27,36):
    driver.get("https://archaeologyofreading.org/viewer/#aor")
    driver.implicitly_wait(5)
    driver.find_element(By.CSS_SELECTOR, "li[data-index-number=\'" + str(pg) + "\']").click()
    
    time.sleep(3)
    book_name = driver.find_element(By.CSS_SELECTOR, "h3.window-manifest-title").text.replace(' ', '').replace(',', '-')
    print(book_name)
    
    flag = True
    counter = 0
    while flag:
        try:
            time.sleep(0.5)

            if len(driver.find_elements(By.CSS_SELECTOR, ".annotationItem")) == 0:
                driver.implicitly_wait(1)
                nextpg = driver.find_element(By.CSS_SELECTOR, ".mirador-osd-next")
                nextpg.click()
            else:
                driver.implicitly_wait(1)
                element = driver.find_element(By.CSS_SELECTOR, ".image-view")

                # create action chain object
                action = ActionChains(driver)

                driver.implicitly_wait(1)
                action.move_to_element(element).context_click().perform()
                pyautogui.press('right')
                pyautogui.press('enter')

                pyautogui.sleep(1)
                keyboard.type(str(counter) + "-" + book_name)
                pyautogui.sleep(1)
                pyautogui.press('enter')

                driver.implicitly_wait(1)
                nextpg = driver.find_element(By.CSS_SELECTOR, ".mirador-osd-next")
                nextpg.click()

        except ElementNotInteractableException as ex:
            print("It is all good, no element there")
            flag = False


        counter += 1




In [ ]:
driver = webdriver.Chrome()

all_metadata = {}
for pg in range(1,36):
    driver.get("https://archaeologyofreading.org/viewer/#aor")
    driver.implicitly_wait(5)
    driver.find_element(By.CSS_SELECTOR, "li[data-index-number=\'" + str(pg) + "\']").click()
    
    time.sleep(1)
    book_name = driver.find_element(By.CSS_SELECTOR, "h3.window-manifest-title").text.replace(' ', '').replace(',', '-')
    
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, ".mirador-icon-metadata-view").click()
    print(book_name)

    time.sleep(3)
    keys = driver.find_elements(By.CSS_SELECTOR, ".metadata-label")
    vals = driver.find_elements(By.CSS_SELECTOR, ".metadata-value")

    book_metadata = {}
    for i, key in enumerate(keys):
        book_metadata[keys[i].text.replace(':','')] = vals[i].text
    
    all_metadata[book_name] = book_metadata

all_metadata

In [ ]:
directory = '/Users/cbrou/Desktop/MRE/aor-pgs/'

for item in all_metadata.items():
    print(item[0])
    attach_data = item[1]
    for filename in os.listdir(directory):
        if item[0] in filename:
            full_img_path = directory + filename

            img = Image.open(full_img_path)

            exifdata = PngInfo()
            exifdata.add_text("UserComment", str(attach_data))

            fn_updated = "/Users/cbrou/Desktop/MRE/aor-pgs/" + filename
            img.save(fn_updated, pnginfo=exifdata)
            


In [ ]:
# an oops revision after accidentally using single quotes in my UserComment rather then double quotes --> single quotes invalid JSON formatting 
import re
directory = '/Users/cbrou/Desktop/MRE/aor-pgs/'

for filename in os.listdir(directory):
    if filename.endswith(".png"):
        full_img_path = directory + filename
        img = Image.open(full_img_path)
        imgtxt = img.text
        
        result = re.sub(r"(?<![a-zA-Z])'|'(?![a-zA-Z])", '"', imgtxt['UserComment'])

        exifdata = PngInfo()
        exifdata.add_text("UserComment", result)

        img.save(full_img_path, pnginfo=exifdata)
        print(result)